In [77]:
import numpy as np
import os
from scipy.optimize import leastsq, fsolve
import math
from CijtVel import Cijtvel
from CijKGvrh import CijVRH
from lmfit import minimize, Parameters, report_fit, fit_report, Minimizer,conf_interval,printfuncs
import lmfit
import sys

In [78]:
dens=3.709

workdir = r'C:\Users\jinzhang\OneDrive - University of New Mexico\0.CijCalPy\cubic'
dindir = r'C:\Users\jinzhang\OneDrive - University of New Mexico\0.CijCalPy\cubic\input'
doutdir = r'C:\Users\jinzhang\OneDrive - University of New Mexico\0.CijCalPy\cubic\output'

os.chdir(workdir) 

print 'working directory:', '\n',os.getcwd()

d1 = np.genfromtxt(os.path.join(dindir,'0.dat'), usecols=0)
d2 = np.genfromtxt(os.path.join(dindir,'0.dat'), usecols=1)
d3 = np.genfromtxt(os.path.join(dindir,'0.dat'), usecols=2)

Vp = np.genfromtxt(os.path.join(dindir,'0.dat'), usecols=3)/1000
Vs1 = np.genfromtxt(os.path.join(dindir,'0.dat'), usecols=4)/1000
Vs2 = np.genfromtxt(os.path.join(dindir,'0.dat'), usecols=5)/1000
err = np.zeros(len(d1))
nonzero_value=np.count_nonzero(Vp)+np.count_nonzero(Vs1)+np.count_nonzero(Vs2)

working directory: 
C:\Users\jinzhang\OneDrive - University of New Mexico\0.CijCalPy\cubic


In [105]:
"####################################### leastsq fitting #######################################"
#weighting parameter epsVp, epsVs1, epsVs2. default 1 for Vp Vs1 and Vs2
epsVp = np.power(Vp,0.0)
epsVs1 = np.power(Vs1,0.0)
epsVs2 = np.power(Vs2,0.0)

VVp = np.zeros(len(d1))
VVs1 = np.zeros(len(d1))
VVs2 = np.zeros(len(d1))
# 3D error: Vp, Vs1, Vs2 uncertainty at len(d1) directions
err3d = np.zeros((len(d1), 3))
# final 1D error array dimenion after kicking out all nan (0) values
lenerr = np.count_nonzero(Vp)+np.count_nonzero(Vs1)+np.count_nonzero(Vs2)

para = Parameters()
para.add('C11', value=328.84, min=200, max=400, vary=True)
para.add('C22', value=128.13, min=70, max=200, vary=True)
para.add('C33', value=116.81, min=50, max=200, vary=True)

#fitting
def resvelo(para):
    C11 = para['C11'].value
    C22 = para['C22'].value
    C33 = para['C33'].value
    
    cld=[C11,C11,C11,C22,C22,C22,C33,C33,C33,0,0, 0, 0, 0, 0, 0, 0, 0,0, 0,0] # Cij, c11,c22,c33,c44,c55,c66,c12,c13,c23,c15,c25,c35,c46
    for i in range (0, len(d1)):
        d=[d1[i],d2[i],d3[i]]
        Vt=Cijtvel(cld,d,dens)
        VVp[i]=Vt[0][0]
        VVs1[i]=Vt[0][1]
        VVs2[i]=Vt[0][2]
        #define if there are nan values for Vp or Vs1 or Vs2
        if Vp[i]==0:
            p=0
        else:
            p=1
        if Vs1[i]==0:
            s1=0
        else:
            s1=1
        if Vs2[i]==0:
            s2=0
        else:
            s2=1

        # 5 different senarios for exsistance of Vp/Vs1/Vs2. Assign err from each velocity measurement in a 3D array err3d.
        if p==1 and s1==0 and s2==0:
            err3d[i][0]=abs((VVp[i]-Vp[i])/epsVp[i])
            err3d[i][1]= 0.0
            err3d[i][2]= 0.0
        else:
            if p==1 and s1==1 and s2==0:
                err3d[i][0] = abs((VVp[i]-Vp[i])/epsVp[i])
                err3d[i][1] = min(abs((VVs1[i]-Vs1[i])/epsVs1[i]),abs((VVs2[i]-Vs1[i])/epsVs1[i])) 
                err3d[i][2] = 0.0               
            else:
                if p==1 and s1==1 and s2==1:
                    err3d[i][0] = abs((VVp[i]-Vp[i])/epsVp[i])
                    err3d[i][1] = min(abs((VVs1[i]-Vs1[i])/epsVs1[i]),abs((VVs2[i]-Vs1[i])/epsVs1[i]))
                    err3d[i][2] = min(abs((VVs1[i]-Vs2[i])/epsVs1[i]),abs((VVs2[i]-Vs2[i])/epsVs2[i])) 
                else:
                    if p==0 and s1==1 and s2==0:
                        err3d[i][0] = 0.0
                        err3d[i][1] = min(abs((VVs1[i]-Vs1[i])/epsVs1[i]),abs((VVs2[i]-Vs1[i])/epsVs1[i]))
                        err3d[i][2] = 0.0
                    else:
                        if p==0 and s1==1 and s2==1:
                            err3d[i][0] = 0.0
                            err3d[i][1] = min(abs((VVs1[i]-Vs1[i])/epsVs1[i]),abs((VVs2[i]-Vs1[i])/epsVs1[i]))
                            err3d[i][2] = min(abs((VVs1[i]-Vs2[i])/epsVs1[i]),abs((VVs2[i]-Vs2[i])/epsVs2[i]))                                    
                        else:              
                            print "input error"
                            exit()
                            
#    print err3d
#    print err3d.shape
    err1d0 = np.reshape(err3d, 3*len(d1))
#    print err1d0
    err1d0list = err1d0.tolist()
#    print err1d0list
    err1dt = [j for j in err1d0list if j != 0]
#    print err1dt
    err1d = np.reshape(err1dt, lenerr)
    #print err1d
    
    return err1d

result = minimize(resvelo, para, method='leastsq', full_output = True, epsfcn=1.0e-10, ftol = 1.0e-15, xtol = 1.0e-10, factor = 1.0)
# result = minimize(resvelo, para, method='nelder')
#param epsfcn: jacobian finite difference approximation increment
#xtol: relative error in approximate solution
#ftol: relative error in the desired sum of squares


In [106]:
"####################################### output fitting result #######################################"
# write error report
from scipy.stats import spearmanr
from numpy import cov

report_fit(result.params,modelpars=None, show_correl=True, min_correl=0.01)
report = fit_report(result.params, modelpars=None, show_correl=True, min_correl=0.01)
file = open(os.path.join(workdir,'cubic_fitting result.txt'), "w")
file.write(report)
file.close()

ci = conf_interval(Minimizer(resvelo, para), result)
lmfit.printfuncs.report_ci(ci)
result.params.pretty_print()

[[Variables]]
    C11:  329.729495 +/- 1.25983098 (0.38%) (init = 328.84)
    C22:  128.473390 +/- 0.72541888 (0.56%) (init = 128.13)
    C33:  117.134783 +/- 1.30405914 (1.11%) (init = 116.81)
[[Correlations]] (unreported correlations are < 0.010)
    C(C11, C22) = -0.576
    C(C22, C33) = -0.305
    C(C11, C33) =  0.184
        99.73%    95.45%    68.27%    _BEST_    68.27%    95.45%    99.73%
 C11:  -4.31413  -2.68311  -1.29536 329.72949  +1.29485  +2.68046  +4.30787
 C22:  -2.47149  -1.54287  -0.74325 128.47339  +0.74458  +1.54981  +3.49470
 C33:  -4.47009  -2.78448  -1.33824 117.13478  +1.33675  +2.77853  +4.45433
Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
C11     329.7      200      400     1.26     True     None     None
C22     128.5       70      200   0.7254     True     None     None
C33     117.1       50      200    1.304     True     None     None
